In [51]:
import pyes
import json
import pandas as pd
import seaborn as sns

host = '140.118.155.14:9200'

fids = []
fids.append('115361891807579')  #Domino Pizza
fids.append('203507002993446')  #PChome 24h
fids.append('186034394769383')  #Cathaylife

post_results = []
comment_results = []
like_results = []

def search(fid):
    conn = pyes.es.ES(host)
    
    #id query
    tq = pyes.query.TermQuery(field="fid", value=fid)
    
    #range query (time)
    ESR = pyes.ESRange(field="created_time", from_value="2015-01-01T00:00:00+0000", to_value="2016-01-01T00:00:00+0000", 
                   include_lower=True ,include_upper=False)
    rq = pyes.query.RangeQuery(qrange=ESR)
    
    qsearch = pyes.query.BoolQuery(must=[tq, rq])
    post_result = conn.search(query=qsearch , indices='facebook_nested' , doc_types='post')
    post_results.append(post_result)

    print("fid:", fid)
    print("The number of the posts in 2015:", len(post_result))
    
    
    comment_result = conn.search(query=qsearch , indices='facebook_nested' , doc_types='comment')
    like_result = conn.search(query=qsearch , indices='facebook_nested' , doc_types='like')
    comment_results.append(comment_result)
    like_results.append(like_result)

for i in fids:
    search(i)

pyes object <pyes.es.ES object at 0x000001B3AF00EE10> is being destroyed, but bulk operations have not been flushed. Call force_bulk()!
pyes object <pyes.es.ES object at 0x000001B3AF1B7710> is being destroyed, but bulk operations have not been flushed. Call force_bulk()!
pyes object <pyes.es.ES object at 0x000001B3AF030E80> is being destroyed, but bulk operations have not been flushed. Call force_bulk()!


fid: 115361891807579
The number of the posts in 2015: 364
fid: 203507002993446
The number of the posts in 2015: 1117
fid: 186034394769383
The number of the posts in 2015: 463


In [15]:
def quantile(month_count_list):
    num = {'min':0,'q1':0,'q2':0,'q3':0,'max':0,'upper':0,'lower':0}
    
    month_count.sort()
    qu = np.percentile(num, np.arange(0, 100, 25))
    num['min'] = month_count[0]
    num['q1'] = qu[1]
    num['q2'] = qu[2]
    num['q3'] = qu[3]
    num['max'] = month_count[len(month_count)-1]
    iqr = qu[3] - qu[1]
    upper = qu[3] + 1.5*iqr
    lower = qu[1] - 1.5*iqr
    
    return num

In [16]:
def count_by_month(result):
    month_amount = {'month':['01','02','03','04','05','06','07','08','09','10','11','12'], 'count':[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
    for i in result:
        m = i['created_time'].split('-')[1]
        m_num = int(m)
        month_amount['count'][m_num-1] += 1
    return month_amount

In [17]:
from bokeh.charts import Bar, output_notebook, show
from bokeh.charts.attributes import cat, color
from bokeh.layouts import row

for i in post_results:
    month_amount = count_by_month(i)
    post_bar = Bar(month_amount, values='count', label=cat(columns='month', sort=False), 
                  title="The number of Posts in 2015", legend='top_right', color='blue', 
                  plot_width=400, plot_height=400)
        
    output_notebook()
    show(post_bar)

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

In [44]:
def count_comment(result, maxime):
    month_data = {'month':[], 'comment_count':[]}
    for i in result:
        if maxime != 0:
            if i['comment_count'] < maxime:
                m = i['created_time'].split('-')[1]
                month_data['month'].append(m)
                month_data['comment_count'].append(i['comment_count'])
        else:
            m = i['created_time'].split('-')[1]
            month_data['month'].append(m)
            month_data['comment_count'].append(i['comment_count'])
    return month_data

In [21]:
from bokeh.charts import BoxPlot

for i in post_results:
    month_comment = count_comment(i, 0)
    
    box = BoxPlot(month_comment, values='comment_count', label=cat(columns='month', sort=False), 
              title="The number of comments of each post in 2015", color='month',
              plot_width=400, plot_height=400)
        
    output_notebook()
    show(box)

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

In [47]:
def count_like(result, maxime):
    month_data = {'month':[], 'likes_count':[]}
    for i in result:
        if maxime != 0:
            if i['likes_count'] < maxime:
                m = i['created_time'].split('-')[1]
                month_data['month'].append(m)
                month_data['likes_count'].append(i['likes_count'])
        else:
            m = i['created_time'].split('-')[1]
            month_data['month'].append(m)
            month_data['likes_count'].append(i['likes_count'])
    return month_data

In [24]:
from bokeh.charts import BoxPlot

for i in post_results:
    month_like = count_like(i, 0)
        
    box = BoxPlot(month_like, values='likes_count', label=cat(columns='month', sort=False), 
              title="The number of likes of each post in 2015", color='month',
              plot_width=400, plot_height=400)
    
    output_notebook()
    show(box)

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

In [33]:
#fid1
month_comment = count_comment(post_results[0], 100)

box = BoxPlot(month_comment, values='comment_count', label=cat(columns='month', sort=False), 
              title="The number of comments of each post in 2015", color='month',
              plot_width=400, plot_height=400)
    
output_notebook()
show(box)

Loading BokehJS ...

In [48]:
month_like = count_like(post_results[0], 2000)

    
box = BoxPlot(month_like, values='likes_count', label=cat(columns='month', sort=False), 
              title="The number of likes of each post in 2015", color='month',
              plot_width=400, plot_height=400)

output_notebook()
show(box)

Loading BokehJS ...

In [41]:
#fid2
month_comment = count_comment(post_results[1], 200)

box = BoxPlot(month_comment, values='comment_count', label=cat(columns='month', sort=False), 
              title="The number of comments of each post in 2015", color='month',
              plot_width=400, plot_height=400)
    
output_notebook()
show(box)

Loading BokehJS ...

In [49]:
month_like = count_like(post_results[1], 6000)

box = BoxPlot(month_like, values='likes_count', label=cat(columns='month', sort=False), 
              title="The number of likes of each post in 2015", color='month',
              plot_width=400, plot_height=400)
    
output_notebook()
show(box)

Loading BokehJS ...

In [45]:
#fid3
month_comment = count_comment(post_results[2], 250)

box = BoxPlot(month_comment, values='comment_count', label=cat(columns='month', sort=False), 
              title="The number of comments of each post in 2015", color='month',
              plot_width=400, plot_height=400)
    
output_notebook()
show(box)

Loading BokehJS ...

In [50]:
month_like = count_like(post_results[2], 1000)
    
box = BoxPlot(month_like, values='likes_count', label=cat(columns='month', sort=False), 
              title="The number of likes of each post in 2015", color='month',
              plot_width=400, plot_height=400)
    
output_notebook()
show(box)

Loading BokehJS ...

In [6]:
def status_count(result):
    status_count = {'type':[], 'count':[]}
    for i in result:
        status_count['type'].append(i['status_type'])
        status_count['count'].append(1)
    return status_count

In [52]:
def status_count_comment(result, maxime):
    status_count = {'type':[], 'count':[], 'comment_count':[], 'likes_count':[]}
    for i in result:
        if maxime != 0:
            if i['comment_count'] < maxime:
                status_count['type'].append(i['status_type'])
                status_count['comment_count'].append(i['comment_count'])
        else:
            status_count['type'].append(i['status_type'])
            status_count['comment_count'].append(i['comment_count'])
    return status_count

In [53]:
def status_count_like(result, maxime):
    status_count = {'type':[], 'count':[], 'comment_count':[], 'likes_count':[]}
    for i in result:
        if maxime != 0:
            if i['likes_count'] < maxime:
                status_count['type'].append(i['status_type'])
                status_count['likes_count'].append(i['likes_count'])
        else:
            status_count['type'].append(i['status_type'])
            status_count['likes_count'].append(i['likes_count'])
    return status_count

In [7]:
from bokeh.charts import Bar, BoxPlot, output_notebook, show
from bokeh.charts.attributes import cat, color
from bokeh.layouts import row

for i in post_results:
    status_type = status_count(i)
    bar = Bar(status_type, values='count', label=cat(columns='type', sort=False), 
                  title="The number of Posts in 2015", legend='top_right', color='blue', 
                  plot_width=400, plot_height=400, agg='sum')
    
    status_type_comment = status_count_comment(i, 0)
    box1 = BoxPlot(status_type_comment, values='comment_count', label=cat(columns='type', sort=False), 
              title="The number of comments of each type of post in 2015", color='type',
              plot_width=400, plot_height=400)
    
    status_type_like = status_count_like(i, 0)
    box2 = BoxPlot(status_type_like, values='likes_count', label=cat(columns='type', sort=False), 
              title="The number of likes of each type of post in 2015", color='type',
              plot_width=400, plot_height=400)
           
    output_notebook()
    show(bar)
    show(row(box1, box2))

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

In [54]:
#fid1
status_type_comment = status_count_comment(post_results[0], 1000)
box1 = BoxPlot(status_type_comment, values='comment_count', label=cat(columns='type', sort=False), 
              title="The number of comments of each type of post in 2015", color='type',
              plot_width=400, plot_height=400)
    
status_type_like = status_count_like(post_results[0], 2000)
box2 = BoxPlot(status_type_like, values='likes_count', label=cat(columns='type', sort=False), 
              title="The number of likes of each type of post in 2015", color='type',
              plot_width=400, plot_height=400)
           
output_notebook()
show(row(box1, box2))

TypeError: Input of table-like dict must be column-oriented.

In [ ]:
#fid2
status_type_comment = status_count_comment(post_results[1], 500)
box1 = BoxPlot(status_type_comment, values='comment_count', label=cat(columns='type', sort=False), 
              title="The number of comments of each type of post in 2015", color='type',
              plot_width=400, plot_height=400)
    
status_type_like = status_count_like(post_results[1], 8000)
box2 = BoxPlot(status_type_like, values='likes_count', label=cat(columns='type', sort=False), 
              title="The number of likes of each type of post in 2015", color='type',
              plot_width=400, plot_height=400)
           
output_notebook()
show(row(box1, box2))

In [ ]:
#fid3
status_type_comment = status_count_comment(post_results[2], 500)
box1 = BoxPlot(status_type_comment, values='comment_count', label=cat(columns='type', sort=False), 
              title="The number of comments of each type of post in 2015", color='type',
              plot_width=400, plot_height=400)
    
status_type_like = status_count_like(post_results[2], 500)
box2 = BoxPlot(status_type_like, values='likes_count', label=cat(columns='type', sort=False), 
              title="The number of likes of each type of post in 2015", color='type',
              plot_width=400, plot_height=400)
           
output_notebook()
show(row(box1, box2))

In [ ]:
%matplotlib inline

import pandas as pd
import seaborn as sns

def boxPlotFBFansPageComp(fids,host,pngFilePath,outlierFilter):
    data=[]
    for fid in fids:
        conn = pyes.es.ES(host)
        tq = pyes.query.TermQuery(field="fid", value=fid)
        tagg = pyes.aggs.TermsAgg('pid', field= 'fid_pid')
        qsearch = pyes.query.Search(tq) 
        qsearch.agg.add(tagg) 
        result = conn.search(query=qsearch , indices='facebook_nested' , doc_types='comment')
        #print json.dumps(result.aggs,indent=2) 
        for fidpid_summary in result.aggs["pid"]["buckets"]:
            #print fidpid_summary["key"] + ',' + str(fidpid_summary["doc_count"])
            if fidpid_summary["doc_count"] < outlierFilter or outlierFilter < 0:
                data.append([fidpid_summary["doc_count"],fid,fidpid_summary["key"]])

    df=pd.DataFrame(data,columns={'count','fid','fidpid'})
    sns.set(style="ticks")
    sns_plot=sns.boxplot(x="fid", y="count", data=df, palette="PRGn")
    fig=sns_plot.get_figure()
    fig.savefig(pngFilePath)

In [ ]:
#output box plot PNG figure
pngFilePath="output.png"

#outlier threshold you want to filter (if you give minus ONE, none of outlier is filtered.)
outlierFilter=2000

#obtain the result and save as "pngFilePath" given from you
boxPlotFBFansPageComp(fids,host,pngFilePath,outlierFilter)

In [ ]:
import datetime
import matplotlib.pyplot as plt


def boxPlotNumofPostPerMonth(fid,host,lowerbound,upperbound,pngFilePath):
    conn = pyes.es.ES(host)
    tq = pyes.query.TermQuery(field="fid", value=fid)
    #tagg = pyes.aggs.TermsAgg('pid', field= 'fid_pid')
    DHAgg = pyes.aggs.DateHistogramAgg('month' ,field='created_time', interval='month',sub_aggs=[])
    tagg = pyes.aggs.TermsAgg('pid', field= 'fid_pid')
    DHAgg.sub_aggs.append(tagg) 
    qsearch = pyes.query.Search(tq) 
    qsearch.agg.add(DHAgg) 
    result = conn.search(query=qsearch , indices='facebook_nested' , doc_types='comment')

    data=[]
    for r in result.aggs["month"]["buckets"]:
        for pid in r["pid"]["buckets"]:
            monthyear=datetime.datetime.strptime(r["key_as_string"], "%Y-%m-%dT%H:%M:%S.%fZ").strftime('%Y/%m')
            data.append([pid["doc_count"],monthyear,pid["key"],r["key"]])

    df=pd.DataFrame(data,columns={'date','count','fid','pid'})
    sns.set(style="ticks")
    sns_plot=sns.boxplot(x="date", y="count", data=df, palette="PRGn")
    fig=sns_plot.get_figure()
    plt.ylim(lower_bound, upper_bound)
    fig.savefig(pngFilePath)

In [ ]:
host = '114.32.24.166:9200'
lower_bound = 0
upper_bound = 500
fid="1448357445384692"
pngFilePath="output3.png"
boxPlotNumofPostPerMonth(fid,host,lower_bound,upper_bound,pngFilePath)

In [ ]:
lower_bound = 0
upper_bound = 500
fid="179080912217031"
pngFilePath="output4.png"
boxPlotNumofPostPerMonth(fid,host,lower_bound,upper_bound,pngFilePath)